In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 32 * 32, 512)  # Adjust input size based on your image dimensions
        self.fc2 = nn.Linear(512, 2)  # 2 classes: melanoma or not

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)  # Adjust input size based on your image dimensions
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Image transformations
transform = transforms.Compose([
    #transforms.Resize((224, 224)),  # Resize images to match CNN input size
    transforms.ToTensor()
])

# Load data
folder = ImageFolder('train', transform=transform)
loader = DataLoader(folder, batch_size=8, shuffle=True)


# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model and move it to GPU
model = CNN().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
total_epochs = 10
for epoch in range(total_epochs):
    correct = 0
    total = 0
    for images, labels in loader:
        # Move input data and labels to the same device as the model
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{total_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

# Save your model if needed
torch.save(model.state_dict(), 'cnn_model.pth')